#### 텐서플로우로 간단한 CNN 만들기

* MNIST 데이터 셋 사용

In [4]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

tf.__version__

'1.2.1'

In [5]:
# 데이터셋 불러오기

# 데이터셋을 읽어올 때 one_hot 인코딩 형태로, 이미지 모양은 그대로 받기 위해 reshape=False를 해줌
# 이미지 사이즈는 28*28*1
mnist = input_data.read_data_sets('MNIST_data', one_hot=True, reshape=False)

Extracting MNIST_data\train-images-idx3-ubyte.gz
Extracting MNIST_data\train-labels-idx1-ubyte.gz
Extracting MNIST_data\t10k-images-idx3-ubyte.gz
Extracting MNIST_data\t10k-labels-idx1-ubyte.gz


## CNN모델 설정 및 Placeholder 설정


In [11]:
X = tf.placeholder(tf.float32, shape = [None, 28, 28, 1])
Y_Label = tf.placeholder(tf.float32, shape=[None, 10])


* Convolution Layer 만들기
   - Conv2D -> Relu -> MaxPool

In [33]:
# 필터 사이즈 (4,4,1) 4개 사용, tf.truncated_normal()로 초기화
# tf.Variable()로 변수 만듬

Kernel1 = tf.Variable(tf.truncated_normal(shape=[4,4,1,4], stddev=0.1))
Bias1 = tf.Variable(tf.truncated_normal(shape=[4], stddev=0.1))
Conv1 = tf.nn.conv2d(X, Kernel1, strides=[1,1,1,1], padding='SAME') + Bias1
Activation1 = tf.nn.relu(Conv1)
Pool1 = tf.nn.max_pool(Activation1, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')

Kernel2 = tf.Variable(tf.truncated_normal(shape=[4,4,4,8], stddev=0.1))
Bias2 = tf.Variable(tf.truncated_normal(shape=[8], stddev=0.1))
Conv2 = tf.nn.conv2d(Pool1, Kernel2,strides=[1,1,1,1], padding='SAME') + Bias2
Activation2 = tf.nn.relu(Conv2)
Pool2 = tf.nn.max_pool(Activation2, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')

In [34]:
## Fully_Connected Layer 만들기


In [36]:
## 8*7*7개의 입력을 받아서 10개의 output을 가지겠다
W1 = tf.Variable(tf.truncated_normal(shape=[8*7*7, 10]))
B1 = tf.Variable(tf.truncated_normal(shape=[10]))
Pool2_flatten = tf.reshape(Pool2, [-1, 8*7*7]) # -1 은 배치 사이즈
OutputLayer = tf.matmul(Pool2_flatten, W1) + B1

In [37]:
## Loss Function과 Opimizer 설정, 그리고 run

In [41]:
# labels = 실제 클래스 값, logits = 출력값
Loss =tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=Y_Label,
                                                             logits=OutputLayer))
# Loss를 최적화알고리즘을 사용하여 Loss를 최소화 시킴, 0.005는 leaning_rate
train_step = tf.train.AdamOptimizer(0.005).minimize(Loss)

# 출력과 실제 클래스가 맞는지 확인
correct_prediction = tf.equal(tf.argmax(OutputLayer, 1), tf.argmax(Y_Label, 1))

# 맞는지 틀린지 모아둔 값들을 평균을 냄
accurary = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

with tf.Session() as sess:
    print("Start -----> ")
    sess.run(tf.global_variable_initializer()) # 변수들을 꼭 초기화 시킴
    for i in range(1000):
        trainingData, Y = mnist.train.next_batch(64)
        sess.run(train_step, feed_dict = {X:trainingData, Y_Label:Y})
        if 1%100:
            print(sess.run(accuarcy, 
                           feed_dict={X:mnist.test.images, 
                                      Y_Label:mnist.test.labels}))

Start -----> 


AttributeError: module 'tensorflow' has no attribute 'global_variable_initializer'